In [1]:
import fonction2 as f
import numpy as np
import os 
from IPython.display import display, Audio
import time as time
import pickle
from scipy.io.wavfile import write
import matplotlib.pyplot as plt
from pyDOE import lhs


In [2]:
def latin_hypercube_sample(n, dim, bounds):
    """
    Generate a Latin Hypercube sample of size n in dim dimensions, with bounds on each dimension.
    :param n: Number of samples to generate.
    :param dim: Number of dimensions.
    :param bounds: List of tuples specifying the lower and upper bounds for each dimension.
    """
    # Normalize the bounds
    normalized_bounds = [(b[0] - (b[1] - b[0]) / 2, (b[1] - b[0]) / 2) for b in bounds]

    # Generate the Latin Hypercube sample
    samples = lhs(dim, samples=n, criterion='center')

    # Scale and translate the samples to the correct bounds
    for i in range(dim):
        samples[:, i] = samples[:, i] * normalized_bounds[i][1] + normalized_bounds[i][0]

    return samples


In [3]:
##corde article: 

T_min = 73.9 - 0.5
T_max = 73.9 + 0.5

rho_l_min = (3.61 - 0.05) * 10**(-3)
rho_l_max = (3.61 + 0.05) * 10**(-3)

Lc_min = 0.65 - 0.01
Lc_max = 0.65 + 0.01 

r_min = (0.40 - 0.02) * 10e-4
r_max = (0.40 + 0.02) * 10e-4

masseC_min = rho_l_min * Lc_min
masseC_max = rho_l_max * Lc_max

I_min =  masseC_min * r_min ** 2 / 2
I_max = masseC_max * r_max **2 / 2

B_min =  (4 - 0.02) * 10**(-5) 
B_max =  (4 + 0.02) * 10**(-5)

E_corde_min = B_min / I_min
E_corde_max = B_max / I_max


In [ ]:
##corde article 2 E4 : https://euphonics.org/wp-content/uploads/2022/03/Guitar_II.pdf

T_min = 70.3 - 0.5
T_max = 70.3 + 0.5

rho_l_min = 0.38 - 0.05
rho_l_max = 0.38 + 0.05

Lc_min = 0.65 - 0.01
Lc_max = 0.65 + 0.01 

r_min = (0.40 - 0.02) * 10e-4
r_max = (0.40 + 0.02) * 10e-4

masseC_min = rho_l_min * Lc_min
masseC_max = rho_l_max * Lc_max

I_min =  masseC_min * r_min ** 2 / 2
I_max = masseC_max * r_max **2 / 2

B_min =  (4 - 0.02) * 10**(-5) 
B_max =  (4 + 0.02) * 10**(-5)

E_corde_min = B_min / I_min
E_corde_max = B_max / I_max

In [4]:
#table composite :

masseT_min = (236e-3 - 0.0003)
masseT_max = (236e-3 + 0.0003)

L_xmin = 0.40 + 0.01
L_xmax = 0.40 + 0.01

L_ymin = 0.259 - 0.002 #largeur de la table (m)
L_ymax= 0.259 + 0.002

h_min = 2.8e-3 - 0.001e-3
h_max = 2.8e-3 + 0.001e-3

rhoT_min = masseT_min / (L_xmax * L_ymax * h_max)  #masse volumique composite
rhoT_max = masseT_max / (L_ymin * h_min * L_xmin)  #masse volumique composite

nu_min = 0.2 - 0.0003
nu_max = 0.2 + 0.0003

ET_min = 7e9 - 0.0005e-9
ET_max = 7e9 + 0.0005e-9


In [5]:
#table acier :

masseT_min = (1588e-3 - 0.003)
masseT_max = (1588e-3 + 0.003)

L_xmin = 0.40 + 0.01
L_xmax = 0.40 + 0.01

L_ymin = 0.259 - 0.002 #largeur de la table (m)
L_ymax= 0.259 + 0.002

h_min = 2.8e-3 - 0.001e-3
h_max = 2.8e-3 + 0.001e-3

rhoT_min = masseT_min / (L_xmax * L_ymax * h_max)  
rhoT_max = masseT_max / (L_ymin * h_min * L_xmin)  

nu_min = 0.2 - 0.0003
nu_max = 0.2 + 0.0003

ET_min = 7e9 - 0.0005e-9
ET_max = 7e9 + 0.0005e-9


In [6]:
N_point = 10

bounds = [(T_min,T_max),(rho_l_min,rho_l_max),(Lc_min,Lc_max), ( E_corde_min,E_corde_max),(I_min,I_max),(h_min,h_max),(nu_min,nu_max),
           (ET_min,ET_max),(rhoT_min,rhoT_max),(L_xmin,L_xmax),(L_ymin,L_ymax)]

XXX = latin_hypercube_sample(N_point,11,bounds)

In [14]:
def creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,nom_fichier):
    
    Fe = 20000
    MS, MS_inv,CS, KS, phiS_Nx_NmS, NmS, xS = f.corde(T,rho_l,L,E_corde,I) 
    tic = time.time()
    # MB_ar,CB_ar,KB_ar,phiB_NxNy_NmB_ar,NmB_ar = f.plaque_article()
    MB_mod, MB_mod_inv, CB_mod, KB_mod, phiB_NxNy_NmB_mod, NmB_mod, x, y= f.plaque_model(h,nu,E_table,rhoT,Lx,Ly)
    tic2 = time.time()
    t, FextS_NxS_Nt = f.Simu_config(xS, Fe, T=3)
    tic3 = time.time()
    M_mod, M_inv, K_mod, C_mod = f.créer_matrice(MS,MB_mod,MS_inv,MB_mod_inv,KS,KB_mod,CS,CB_mod)
    #M_ar,K_ar,C_ar = f.créer_matrice(MS,MB_ar,KS,KB_ar,CS,CB_ar)

    tic4 = time.time()
    W_mod_a2, Z_mod_a2 = f.UK_params(M_mod,M_inv,NmS, NmB_mod, phiS_Nx_NmS, phiB_NxNy_NmB_mod, article = False, model = True, mode = 'A2',x = x, y = y)
    #W_ar_a2,Z_ar_a2 = f.UK_params(M_ar,NmS, NmB_ar, phiS_Nx_NmS, phiB_NxNy_NmB_ar, article = True, model = False, mode = 'A2',x =0, y = 0)

    tic5 = time.time()
    Q, F = f.lounch_simu_article(t,FextS_NxS_Nt,phiS_Nx_NmS,NmS,NmB_mod,M_inv,C_mod,K_mod,Z_mod_a2,W_mod_a2)
    #Q, F = f.lounch_simu_article(t,FextS_NxS_Nt,phiS_Nx_NmS,NmS,NmB_ar,M_ar,C_ar,K_ar,Z_ar_a2,W_ar_a2)
    tic6 = time.time()

    FS = F[:NmS,:]
    #FB = F[NmS:,:]
    FS_NxS_Nt = phiS_Nx_NmS @ FS
    #FB_NxNy_Nt = phiB_NxNy_NmB_mod @ FB
    tic7 = time.time()
    #write("1ier_essai/acier/"+ nom_fichier + '.wav',rate=Fe ,data = FS_NxS_Nt[-1,:] / max(FS_NxS_Nt[-1,:]))
    display(Audio(FS_NxS_Nt[-1,:], rate=Fe))
    print(tic2 - tic)
    print(tic3 - tic2) 
    print(tic4 - tic3)
    print(tic5 - tic4)
    print(tic6 - tic5)
    print(tic7 - tic6)
    print(tic7 - tic)

In [15]:
T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly = XXX[0]
creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,str(0))

Fréquence du dernier mode de plaque calculé : 507 Hz


0.022230148315429688
0.0005071163177490234
0.000102996826171875
0.00017714500427246094
8.593435764312744
0.3406038284301758
8.957056999206543


In [ ]:
for i in range(N_point):
    T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly = XXX[i]
    creer_son(T,rho_l,L,E_corde,I,h,nu,E_table,rhoT,Lx,Ly,str(i))

In [ ]:
dico = {}
for i in range(N_point):
    kle = str(i) + ".wav"
    dico[kle] = {}
    dico[kle]['T_corde'] = XXX[i][0]
    dico[kle]['rho_l_corde'] = XXX[i][1]
    dico[kle]['L_corde'] = XXX[i][2]
    dico[kle]['E_corde'] = XXX[i][3]
    dico[kle]['I_corde'] = XXX[i][4]
    dico[kle]['h_table'] = XXX[i][5]
    dico[kle]['nu_table'] = XXX[i][6]
    dico[kle]['E_table'] = XXX[i][7]
    dico[kle]['rho_table'] = XXX[i][8]
    dico[kle]['Lx_table'] = XXX[i][9]
    dico[kle]['Ly_table'] = XXX[i][10]

with open('1ier_essai/acier/parametres.pickle', 'wb') as f:
    pickle.dump(dico, f)
